# 2020 Advent of Code - Day 4

## Part 1

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

```
byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
```

Passport data is validated in batch files (your puzzle input). 
Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. 
Passports are separated by blank lines.

Here is an example batch file containing four passports:

```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm
```

```
iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929
```

```
hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm
```

```
hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

## Part 1 Solution

In [67]:
reqd_fields = ['byr','iyr','eyr','hgt','hcl','ecl','pid']

In [68]:
with open('./2020_test_files/day4_test.txt') as f:
    test_data = f.read()

Uggh - this makes me shudder with its ugliness

In [78]:
def extract_batch(input='./2020_input_files/day4_input.txt'):
    valid_count = 0
    with open(input) as f:
        batch = []
        for line in f:
            if line == "\n":
                check =  {item.split(':')[0]: item.split(':')[1] for item in batch}
                batch_keys = list(check.keys())
                if all (k in batch_keys for k in reqd_fields):
                    valid_count += 1
                batch = []
            else:
                batch.extend(line.rstrip().split(" "))
        if batch:
            check =  {item.split(':')[0]: item.split(':')[1] for item in batch}
            batch_keys = list(check.keys())
            if all (k in batch_keys for k in reqd_fields):
                valid_count += 1
        return valid_count


In [79]:
extract_batch()

226

### Refactor Attempt

In [80]:
def extract_keyvals(batch):
    return {item.split(':')[0]: item.split(':')[1] for item in batch}

In [83]:
def key_presence_checker(key_vals):
    batch_keys = list(key_vals.keys())
    return all (k in batch_keys for k in reqd_fields)

In [85]:
def valid_password_count(input='./2020_input_files/day4_input.txt'):
    valid_count = 0
    with open(input) as f:
        batch = []
        for line in f:
            if line == "\n":
                if key_presence_checker(extract_keyvals(batch)):
                    valid_count += 1
                batch = []
            else:
                batch.extend(line.rstrip().split(" "))
        if batch:
            if key_presence_checker(extract_keyvals(batch)):
                    valid_count += 1
        return valid_count

In [86]:
valid_password_count()

226

## Part 2

--- Part Two ---
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

Your job is to count the passports where all required fields are both present and valid according to the above rules. 

Here are some invalid passports:

```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```

Here are some valid passports:

```
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022


iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [90]:
import re

In [97]:
def validate_number_range(value: str, max: int, min: int,) -> bool:
    if value.isdigit():
        num = int(value)
        return max >= num >= min
    else:
        return False

In [216]:
def validate_hgt(x: str) -> bool:
    match = re.match(r'^([0-9]+)(cm|in)$',x)
    if match:
        match_groups = match.groups()
        if match_groups[1] == "cm":
            return 150 <= int(match_groups[0]) <= 193
        elif match_groups[1] == "in":
            return 59 <= int(match_groups[0]) <= 76
    else:
        return False
        

In [217]:
def validate_pid(value: str) -> bool:
    match = re.match(r'^([0-9]{9})$',value)
    if match:
        return True
    else:
        return False

In [218]:
passport_rules = {
    'byr': lambda x: validate_number_range(x, max=2002, min=1920),
    'iyr': lambda x: validate_number_range(x, max=2020, min=2010),
    'eyr': lambda x: validate_number_range(x, max=2030, min=2020),
    'hgt': lambda x: validate_hgt(x),
    "hcl": lambda x: x[0] == "#" and all(y in "1234567890abcdef" for y in x[1:]),
    "ecl": lambda x: x in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"],
    "pid": lambda x: validate_pid(x)
}

### Test passport rules

In [219]:
assert passport_rules['byr']('2002') == True
assert passport_rules['byr']('2003') == False
assert passport_rules['hgt']('60in') == True
assert passport_rules['hgt']('190cm') == True
assert passport_rules['hgt']('190in') == False
assert passport_rules['hgt']('190') == False
assert passport_rules['hcl']('#123abc') == True
assert passport_rules['hcl']('#123abz') == False
assert passport_rules['hcl']('123abc') == False
assert passport_rules['ecl']('brn') == True
assert passport_rules['ecl']('wat') == False
assert passport_rules['pid']('000000001') == True
assert passport_rules['pid']('x0123456789') == False
assert passport_rules['pid']('0000000011') == False

In [220]:
def new_passport_checker(x: dict) -> bool:
    if key_presence_checker(x):
        for key in x.keys():
            if key in reqd_fields:
                if passport_rules[key](x[key]) == False:
                    return False
        return True
    else:
        return False

In [221]:
def strict_valid_password_count(input='./2020_input_files/day4_input.txt'):
    valid_count = 0
    with open(input) as f:
        batch = []
        for line in f:
            if line == "\n":
                if new_passport_checker(extract_keyvals(batch)):
                    valid_count += 1
                batch = []
            else:
                batch.extend(line.rstrip().split(" "))
        if batch:
            if new_passport_checker(extract_keyvals(batch)):
                    valid_count += 1
        return valid_count

### Test passport checker

In [222]:
assert strict_valid_password_count(input='./2020_test_files/day4_valid.txt') == 4
assert strict_valid_password_count(input='./2020_test_files/day4_invalid.txt') == 0

In [223]:
strict_valid_password_count()

160